# Analysis of Commercial Real Estate (CRE) Market Index

## Executive Summary

The goal of this analysis was to identify trends in multi-family commercial real estate across regions in the United States. The predictive power of variables were tested and correlations observed.


## The Data

The Atlanta Federal Reserve creates the Commercial Real Estate Market Index (CREMI), a quarterly indicator that tracks overall CRE conditions across roughly 390 U.S. metro areas and their surrounding regions. The index offers a broad look at several property sectors - including office, hospitality, multifamily, and more - giving users a baseline from which to explore specific market trends in greater depth. The Atlanta Fed also publishes a diffusion index that highlights the share of commercial real estate and economic measures exhibiting positive momentum.

The data the Atlanta Federal Reserve uses to create the CREMI are obtained through public and nonpublic data sources. Among the underlying economic factors are occupancy rates, market cap rates, asset values, the unemployment rate, and more.


<table border="1" cellspacing="0" cellpadding="2" width="100%">
  <thead>
    <tr>
      <th colspan="4"><strong>Asset Type: Multifamily</strong></th>
    </tr>
    <tr>
      <th>Variable</th>
      <th>Type</th>
      <th>Transformation</th>
      <th><center>Sign</center></th>
    </tr>
  </thead>

  <tbody>
    <tr>
      <td>Occupancy Rate</td>
      <td>CRE*</td>
      <td>Level</td>
      <td><center>+</center></td>
    </tr>
    <tr>
      <td>Net Operating Income Index</td>
      <td>CRE</td>
      <td>Log Change (YoY)</td>
      <td><center>+</center></td>
    </tr>
    <tr>
      <td>Market Cap Rate</td>
      <td>CRE</td>
      <td>Level</td>
      <td><center>-</center></td>
    </tr>
    <tr>
      <td>Asset Value</td>
      <td>CRE</td>
      <td>Percent Change (YoY)</td>
      <td><center>+</center></td>
    </tr>
    <tr>
      <td>Absorption Units</td>
      <td>CRE</td>
      <td>Levels</td>
      <td><center>+</center></td>
    </tr>
    <tr>
      <td>Unemployment Rate</td>
      <td>Economic</td>
      <td>Level</td>
      <td><center>-</center></td>
    </tr>
    <tr>
      <td>Households</td>
      <td>Economic</td>
      <td>Percent Change (YoY)</td>
      <td><center>+</center></td>
    </tr>
    <tr>
      <td>Industrial Production</td>
      <td>Economic</td>
      <td>Percent Change (YoY)</td>
      <td><center>+</center></td>
    </tr>
    <tr>
      <td>CPI</td>
      <td>Economic</td>
      <td>Percent Change (YoY)</td>
      <td><center>-</center></td>
    </tr>
  </tbody>
  <tfoot>
    <tr>
      <td colspan="4" style="font-size:0.8rem; margin: 0.5rem 0; padding: 4px;">
        *Economic factors for variables with type "CRE" in the table above are proprietary and not publicly available.
      </td>
    </tr>
  </tfoot>
</table>


## Obtain the Data

The data was obtained from the [Atlanta Fed website](https://www.atlantafed.org/research/data-and-tools/commercial-real-estate-market-index?utm_source=chatgpt.com#Tab7).
The data included a CSV file titled CREMI_CBSA_Results.csv with 1,741,070 rows and 7 columns and a size of 157 MB.


In [1]:
import duckdb

con = duckdb.connect()

raw = con.read_csv("datasets/CREMI_CBSA_Results.csv").df()
# con.register("df", df)
print(f"{raw.shape[0]:,} rows and {raw.shape[1]:,} columns")


1,741,070 rows and 7 columns


## Preview the Data


In [2]:
raw.head(5)

,column0,Geography.Name,CBSA.Code,Asset_Type,DT,variable,value
0,1,Abilene - TX,10180,Office,20071231,CREMI,0.700718
1,2,Abilene - TX,10180,Office,20080331,CREMI,0.432477
2,3,Abilene - TX,10180,Office,20080630,CREMI,0.146078
3,4,Abilene - TX,10180,Office,20080930,CREMI,-0.466332
4,5,Abilene - TX,10180,Office,20081231,CREMI,-2.466985


## Reformat the Data

The data in its original form was reformatted to have clearer column names and prepare it for further analysis. Additionally, unnecessary columns were removed. The cleaned data table can be seen below.


In [3]:
import pandas as pd

# Create a copy of the raw data
df = raw.copy() 

# Reformat column names
df.columns = [c.replace(".", "_") for c in df.columns]
df.rename(columns={'DT': 'date'}, inplace=True)

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['date'] = df['date'].dt.date

# Drop unnecessary columns
df.drop('column0', axis=1, inplace=True)
df.head()

,Geography_Name,CBSA_Code,Asset_Type,date,variable,value
0,Abilene - TX,10180,Office,2007-12-31,CREMI,0.700718
1,Abilene - TX,10180,Office,2008-03-31,CREMI,0.432477
2,Abilene - TX,10180,Office,2008-06-30,CREMI,0.146078
3,Abilene - TX,10180,Office,2008-09-30,CREMI,-0.466332
4,Abilene - TX,10180,Office,2008-12-31,CREMI,-2.466985


### Remove Redundancy

Notice that there is a Geography_Name representing the city and state of the region as well as a CBSA_Code representing the numeric code assigned by the U.S. Office of Management and Budget (OMB) that uniquely identifies a Core-Based Statistical Area (CBSA). The Geography_Name and CBSA_Code variables describe the same geographical unit so the CBSA_Code was dropped from the table. For further confirmation of this, the presence of a 1-to-1 matching of the Geography_Name and CBSA_Code was tested and confirmed below.


In [4]:

# Does every Geography_Name have exactly one CBSA_Code and vice versa?
if (df.groupby("Geography_Name")["CBSA_Code"].nunique() == 1).all() and(df.groupby("CBSA_Code")["Geography_Name"].nunique() == 1).all():
    print("True. Every Geography_Name has exactly one CBSA_Code and vice versa.")
    df.drop("CBSA_Code", axis=1, inplace=True)
else: 
    print("False. There is not a 1-to-1 mapping between Geography_Name and CBSA_Code.")

df.head()

True. Every Geography_Name has exactly one CBSA_Code and vice versa.


,Geography_Name,Asset_Type,date,variable,value
0,Abilene - TX,Office,2007-12-31,CREMI,0.700718
1,Abilene - TX,Office,2008-03-31,CREMI,0.432477
2,Abilene - TX,Office,2008-06-30,CREMI,0.146078
3,Abilene - TX,Office,2008-09-30,CREMI,-0.466332
4,Abilene - TX,Office,2008-12-31,CREMI,-2.466985


Next, we determined that there are five unique asset types. Each one contained over 300,000 CREMI values.


In [5]:
df["Asset_Type"].value_counts().map("{:,}".format)


Asset_Type
Multifamily    404,900
Hospitality    374,400
Office         324,730
Industrial     324,670
Retail         312,370
Name: count, dtype: object

In [ ]:
out = con.execute(
    """
    SELECT Geography_Name, value 
    FROM df 
    WHERE Geography_Name = 'Fort Myers - FL'
    """).fetchdf()
print(out)